In [373]:
from rdflib import Graph
import pprint

student1 = Graph()
student1.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\s1.rdf")
student2 = Graph()
student2.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\s2.rdf")
student3 = Graph()
student3.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\s3.rdf")
student4 = Graph()
student4.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\s4.rdf")


proposal = Graph()
proposal.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\proposedPlan.rdf")

HEC = Graph()
HEC.parse(r"C:\Users\sarim\OneDrive\Desktop\KRR PROJECT FINAL\csvs\hec.rdf")

<Graph identifier=N53613deb532844978b2c4b01166e113f (<class 'rdflib.graph.Graph'>)>

In [119]:
proposal_info = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?sem ?value
WHERE {
    ?object plan:courseTitle ?value.
    ?object plan:semester ?sem.
    
} 
ORDER BY ASC(?sem)
"""


qres = proposal.query(proposal_info)
for row in qres:
    print(f"{row.sem}: {row.value}")
    

1: Introduction to Info. & Comm. Technologies
1: Applied Physics
1: Programming Fundamentals Lab
1: English Composition & Comprehension
1: Calculus & Analytical Geometry
1: Programming Fundamentals
2: Object Oriented Programming Lab
2: University Elective
2: Digital Logic Design
2: Digital Logic Design Lab
2: Communication & Presentation Skills
2: Probability & Statistics
2: Object Oriented Programming
3: Data Structures & Algorithms Lab
3: Comp. Organization & Assembly Language
3: Professional Practices
3: Comp. Organization & Assembly Language Lab
3: CS supporting
3: Data Structures & Algorithms
4: University Elective
4: Linear Algebra
4: Theory of Automata
4: Database Systems
4: Database Systems Lab
4: Design & Analysis of Algorithms
5: CS Supporting
5: CS Supporting
5: Operating Systems
5: Operating Systems Lab
5: Software Engineering
5: Compiler Construction
6: Artificial Intelligence Lab
6: CS Elective
6: CS Elective
6: Computer Networks
6: Technical & Business Writing
6: Artific

# Main Queries

In [266]:
#explore query result
#qres.bindings


## Degree Eligibility Criteria

In [286]:
#check if student has covered required credit_hour


student_ch_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX Std-003:<file:///csvs/Std-003.csv#>
SELECT (IF(sum(xsd:integer(?value)) < 130, 0, 1) as ?result)
WHERE {
    ?subject Std-003:creditHour ?value.
    ?subject Std-003:points ?p.
    FILTER(xsd:float(?p)>=1.0).
} """

qres = student3.query(student_ch_query)
for row in qres:
    print(f"Has student covered minimum credithour: {'Yes' if row.result.toPython() == 1 else 'No'}")


Has student covered minimum credithour: Yes


## Missing Courses of Student from HEC plan

In [374]:

#Check if student has missed any courses in cc

q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX p20plan: <file:///csvs/HEC%20plan.csv#>
PREFIX Std-003:<file:///csvs/Std-003.csv#>


SELECT ?result ?group ?credit
WHERE {
  ?object1 p20plan:courseTitle ?result.
  MINUS {
    ?object Std-003:courseName ?result.
  }
  ?object1 p20plan:courseGroup ?group.
  ?object1 p20plan:creditHour ?credit.
  FILTER (?group = 'Computing - Core').
}
"""

qres=(student3+HEC).query(q)
print("Missed Computer Science Core Compulsory Courses by Student 3")
for row in qres:
    print(f"{row.result}: {row.credit} credit hour")

Missed Computer Science Core Compulsory Courses by Student 3
Data Structures & Algorithms Lab: 1 credit hour


## Missing Pre-requisites

In [302]:
prereq_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT DISTINCT ?value ?value2 ?value3 ?sem
WHERE {
    
    ?object Std-002:courseName ?value.
    ?object Std-002:semester ?sem.
    ?object2 plan:courseTitle ?value.
    ?object2 plan:preRequisite ?value2.
    OPTIONAL {
    ?object2 plan:preRequisite ?value3.
    ?object3 Std-002:courseName ?value3.
    }
    
    
    
} """

qres = (student2+proposal).query(prereq_query)
for row in qres:
    if row.value3 == None:
        print(f"In semester {row.sem} {row.value} has prerequisite {row.value2} which student didn't take.")

In semester 5 Database Systems Lab has prerequisite Data Structures & Algorithms Lab which student didn't take.


## FYP Eligibility

In [391]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX p20plan: <file:///csvs/HEC%20plan.csv#>
PREFIX Std-003:<file:///csvs/Std-003.csv#>

SELECT (IF(sum(xsd:integer(?crHr)) < 33, 0, 1) as ?result)
WHERE {
    ?subject Std-003:courseName ?stdCourse.
    ?subject Std-003:points ?stdPoints.
    ?subject Std-003:creditHour ?crHr.
    ?subject1 p20plan:courseTitle ?stdCourse.
    ?subject1 p20plan:courseGroup ?group.
    
    FILTER (?group = 'CS Core' && xsd:float(?stdPoints) >= 1.0).
    
} 
"""
csCoreList=[]
qres = (student3+HEC).query(query)
for row in qres:
    #print(f"{row.stdCourse}:    {row.group}:    {row.stdPoints}")
    print(f"If student is eligible to take FYP: {'Yes' if row.result.toPython() == 1 else 'No'}")

If student is eligible to take FYP: No


## University Proposal Plan Validation According to HEC assigned Credit Hours

In [398]:
#Query : Verification of University proposed plan

query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX plan: <file:///csvs/plan.csv#>
PREFIX p20plan: <file:///csvs/HEC%20plan.csv#>


SELECT ?cgroup  (((SUM(xsd:integer(?crhr)))=(SUM(xsd:integer(?heccrhr)))) as ?groupcr)
WHERE {
    {
    ?subject plan:creditHours ?crhr.
    ?subject2 p20plan:creditHour ?heccrhr.
    ?subject2 p20plan:courseGroup ?cgroup.
    ?subject plan:courseTitle ?course.
    ?subject2 p20plan:courseTitle ?course.
    FILTER(?cgroup != "University Elective")
    FILTER(?course != "Final Year Project")

    }

} 
group by ?cgroup


"""

qres = (HEC+proposal).query(query)
print("Course Group of courses in University Proposal Fullfilling HEC criteria\n")


for row in qres:
    print(f"{row.cgroup.toPython()}: {row.groupcr.toPython()}")




Course Group of courses in University Proposal Fullfilling HEC criteria

General Education: True
CS Core: True
Mathematics & Science Foundation: True
CS Supporting: True
Computing - Core: True
CS Elective: True


## Student Covering Courses According to Hec Course Group

In [365]:
knows_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?value2 (SUM(xsd:integer(?value3))AS ?sum)
WHERE {
    ?object p20plan:courseTitle ?value.
    ?object p20plan:courseGroup ?value2.
    ?object2 Std-002:courseName ?value.
    ?object2 Std-002:creditHour ?value3.
    FILTER (xsd:float(?value3) >= 1.0)
} 

GROUP BY ?value2
"""

qres = (HEC+student2).query(knows_query)
i=1
for row in qres:
    None
    print(f"{i}-> {row.value2}: {row.sum} credit hours")
    i+=1

1-> General Education: 18 credit hours
2-> CS Core: 17 credit hours
3-> Mathematics & Science Foundation: 10 credit hours
4-> University Elective: 12 credit hours
5-> Computing - Core: 32 credit hours
6-> CS Elective: 15 credit hours
7-> CS Supporting: 3 credit hours


## Courses failed by Student

In [357]:
#Query to find courses where student failed

student_ch_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX Std-001:<file:///csvs/Std-001.csv#>
SELECT ?cours ?grade
WHERE {
    ?x Std-001:courseName ?cours.
    ?x Std-001:points ?gp.
    ?x Std-001:grade ?grade .
    FILTER (?gp < "1")
} """

print("Student 1 failed in these courses")
qres = student1.query(student_ch_query)
for row in qres:
    print(f"{row.cours}: {row.grade} ")


Student 1 failed in these courses
CS Elective: S 
Artificial Intelligence: S 
Technical & Business Writing: S 


## Dean's List eligibility Semester wise

In [389]:
#Query : Calculating the students semester-wise dean's list

query1 ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX Std-003:<file:///csvs/Std-003.csv#>

SELECT ?sem (SUM((xsd:float(?gpa)*(xsd:integer(?crhr))))/SUM(xsd:integer(?crhr)) as ?avggpa)
WHERE {
    ?subject Std-003:points ?gpa.
    ?subject Std-003:creditHour ?crhr.
    ?subject Std-003:semester ?sem.
} 
group by ?sem
order by ?sem
"""

courses = []
qres = student3.query(query1)
for row in qres:
    #print(f"{row.sem.toPython()} : {row.avggpa.toPython()}")
    print(f"Student 3 eligiblity for the Dean's List in Semester {row.sem.toPython()}: {'Yes' if row.avggpa.toPython() >= 3.5 else 'No'}")

Student 3 eligiblity for the Dean's List in Semester 1: No
Student 3 eligiblity for the Dean's List in Semester 2: No
Student 3 eligiblity for the Dean's List in Semester 3: No
Student 3 eligiblity for the Dean's List in Semester 4: Yes
Student 3 eligiblity for the Dean's List in Semester 5: Yes
Student 3 eligiblity for the Dean's List in Semester 6: Yes
Student 3 eligiblity for the Dean's List in Semester 7: Yes
Student 3 eligiblity for the Dean's List in Semester 8: Yes
